In [5]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [8]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [10]:
# Your code
data.head(5)

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0


In [44]:
X, y= data["text"], data["label"]

## Data Preprocessing

In [17]:
import string
from nltk.corpus import stopwords
import nltk
#nltk.download("stopwords")

print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [ ]:
import re

# <>! AI GENERATED PIPELINE <>!
def clean_html(text):
  text= re.sub(r"<script.*?>.*?</script>", "", text, flags=re.S) # remove html
  text= re.sub(r"<style.*?>.*?</style>", "", text, flags=re.S)   # remove jss
  text= re.sub(r"<!--.*?-->", "", text, flags=re.S)              # remove css
  text= re.sub(r"<[^>]+>", "", text)                             # remove comments
  text= re.sub(r"[^a-zA-Z\s]", " ", text)                        # remove especial ch
  text= re.sub(r"\d+", "", text)                                 # remove numbers
  text= re.sub(r"\b[a-zA-Z]\b", "", text)                        # remove single ch
  text= re.sub(r"^\s+[a-zA-Z]\s+", "", text)                     # remove single ch from start
  text= re.sub(r"\s+", " ", text)                                # substitute multiple spaces with single space
  text= re.sub(r"^b\s+", "", text)                               # remove prefixed 'b'
  text= text.lower()                                             # convert to lowercase

  return text.strip()

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [27]:
# Your code
data["cleaned_text"]= data["text"].apply(clean_html)
data.head(5)

,text,label,cleaned_text
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal am...
1,Will do.,0,will do
2,Nora--Cheryl has emailed dozens of memos about...,0,nora cheryl has emailed dozens of memos about ...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sir fmadam know that this proposal might ...
4,fyi,0,fyi


## Now let's work on removing stopwords
Remove the stopwords.

In [29]:
# Your code
from nltk.tokenize import word_tokenize

stop_words= set(stopwords.words("english"))
no_stopwords_list= []

for text in data["cleaned_text"]:
  tokens = word_tokenize(text)
  filtered_tokens = []
  for t in tokens:
    if t.lower() not in stop_words:
      filtered_tokens.append(t)
  no_stopwords_list.append(" ".join(filtered_tokens))

data["no_stopwords"] = no_stopwords_list
data.head()

,text,label,cleaned_text,no_stopwords
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal am...,dear sir strictly private business proposal mi...
1,Will do.,0,will do,
2,Nora--Cheryl has emailed dozens of memos about...,0,nora cheryl has emailed dozens of memos about ...,nora cheryl emailed dozens memos haiti weekend...
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sir fmadam know that this proposal might ...,dear sir fmadam know proposal might surprise e...
4,fyi,0,fyi,fyi


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [30]:
# Your code

nltk.download("wordnet")
nltk.download("omw-1.4")
nltk.download("punkt") 

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [40]:
from nltk.stem import WordNetLemmatizer
lemmatizer= WordNetLemmatizer()

lemmatized_list= []

for text in data["no_stopwords"]:
  tokens= word_tokenize(text)
  lemm= []
  for t in tokens:
    lemm.append(lemmatizer.lemmatize(t))
  lemmatized_list.append(" ".join(lemm))

data["lemmatized"]= lemmatized_list
data.head()

,text,label,cleaned_text,no_stopwords,lemmatized,money_mark,suspicious_words,text_len
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal am...,dear sir strictly private business proposal mi...,dear sir strictly private business proposal mi...,1,1,1486
1,Will do.,0,will do,,,0,0,0
2,Nora--Cheryl has emailed dozens of memos about...,0,nora cheryl has emailed dozens of memos about ...,nora cheryl emailed dozens memos haiti weekend...,nora cheryl emailed dozen memo haiti weekend p...,0,0,111
3,Dear Sir=2FMadam=2C I know that this proposal ...,1,dear sir fmadam know that this proposal might ...,dear sir fmadam know proposal might surprise e...,dear sir fmadam know proposal might surprise e...,1,1,1346
4,fyi,0,fyi,fyi,fyi,0,0,3


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [ ]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

vectorizer= CountVectorizer()
X_bow= vectorizer.fit_transform(data["lemmatized"])
feature_names= vectorizer.get_feature_names_out()

spam_idx= data[data["label"]==1].index
ham_idx=  data[data["label"]==0].index

spam_bow= X_bow[spam_idx, :]
ham_bow=  X_bow[ham_idx, :]

spam_counts= pd.DataFrame(spam_bow.toarray(), columns=feature_names).sum().sort_values(ascending=False)
ham_counts=  pd.DataFrame(ham_bow.toarray(), columns=feature_names).sum().sort_values(ascending=False)

print("Top 10 words in SPAM:")
print(spam_counts.head(10))
print("\nTop 10 words in HAM:")
print(ham_counts.head(10))

Top 10 words in SPAM:
money          981
account        895
bank           800
fund           781
transaction    549
business       514
country        508
mr             489
nbsp           475
million        460
dtype: int64

Top 10 words in HAM:
state        136
pm           127
would        107
president     99
time          95
call          94
mr            91
obama         84
percent       81
secretary     79
dtype: int64


## Extra features

In [ ]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
# im not gonna change all my code fk off, explain this sht better from beginning this is annoying
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

---

In [37]:
money_simbol_list= "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words=  "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

In [38]:
data['money_mark']=       data['lemmatized'].str.contains(money_simbol_list, case=False, regex=True)*1
data['suspicious_words']= data['lemmatized'].str.contains(suspicious_words, case=False, regex=True)*1
data['text_len']=         data['lemmatized'].apply(lambda x: len(x))

In [39]:
data.head(3)

,text,label,cleaned_text,no_stopwords,lemmatized,money_mark,suspicious_words,text_len
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1,dear sir strictly private business proposal am...,dear sir strictly private business proposal mi...,dear sir strictly private business proposal mi...,1,1,1486
1,Will do.,0,will do,,,0,0,0
2,Nora--Cheryl has emailed dozens of memos about...,0,nora cheryl has emailed dozens of memos about ...,nora cheryl emailed dozens memos haiti weekend...,nora cheryl emailed dozen memo haiti weekend p...,0,0,111


## How would work the Bag of Words with Count Vectorizer concept?

In [ ]:
# Your code

The `BoW` with CountVectorizer is a way to convert text into numbers that a machine learning model can understand. Simply, this will scan allt he text and list all unique words (tokens) where each word becomes a unique feature. For each document it counts how many times each word from the vocabulary appears and represent it as a m atrix

In [43]:
testo= [
  "I love NLP",
  "NLP is fun",
  "I love coding"
]

vectorizer= CountVectorizer()
_X= vectorizer.fit_transform(testo)

print(vectorizer.get_feature_names_out())
print(_X.toarray())

['coding' 'fun' 'is' 'love' 'nlp']
[[0 0 0 1 1]
 [0 1 1 0 1]
 [1 0 0 1 0]]


## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [ ]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer #gonna use tf id vectorizer

tfidf_vectorizer= TfidfVectorizer() # load the vectorizer
X_vectorized= tfidf_vectorizer.fit_transform(data["lemmatized"]) #vectorize df

print("Shape of TF-IDF matrix:", X_vectorized.shape) # shape of X_vectorized

Shape of TF-IDF matrix: (1000, 29929)


## And the Train a Classifier?

In [ ]:
# Your code
# ???????? i don't get it

### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [ ]:
# Your code